In [ ]:
from pyspark.sql import Row
from xinghe.spark import *
from app.common.json_util import *
from xinghe.s3 import *

config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
}
from pyspark.sql.types import StructType, StructField, StringType
import re
import hashlib
from lxml.etree import HTML

MAX_OUTPUT_ROW_SIZE = 1024 * 1024 * 1024 * 1.5
DUMPS = [
    ...
]

CC_WARC = 's3://xxx/'
output_path = "s3://xxx/"
spark = new_spark_session("cc_dumps.dedup.fir", config)
sc = spark.sparkContext

# html source

In [ ]:
# 获取 cc warc path list
warc_paths = []
for dump in DUMPS:
    dump_path = f'{CC_WARC}{dump}/'
    warc_paths.extend([x for x in list(list_s3_objects(dump_path, recursive=True)) if "/warc/" in x])


In [ ]:
def html_to_content(html_str: str, url: str) -> str:
    if html_str.strip() and isinstance(html_str,str):
        html_str = re.sub(r'<\?[^>]*\?>', '', html_str.strip())
        try:
            html_etree = HTML(html_str)
        except:
            return None
        if html_etree:
            for element in html_etree.xpath('//*[self::script or self::style]'):
                element.getparent().remove(element)
            text = ''.join(html_etree.xpath("//text()"))
            cleaned_text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)
            cleaned_text = re.sub(r'\s+', '', cleaned_text).strip()
            return sha256_hash(cleaned_text)

def sha256_hash(string):
    return hashlib.sha256(string.encode()).hexdigest()
    
def parse_path_to_html(iter):
    seen = set()
    for fpath in iter:
        for zz in read_s3_rows(fpath):
            detail_datas = json_loads(zz.value)
            hash_html = html_to_content(detail_datas.get("html"), detail_datas["url"]) if detail_datas.get("html", "") else None
            if hash_html and hash_html not in seen:
                seen.add(hash_html)
                line = {
                    "sub_path": fpath.split('/')[4],
                    "hash_html": hash_html,
                    "track_id": detail_datas["track_id"],
                }
                yield Row(**{"value": json_dumps(line)})

In [ ]:
# mapPartitions 对 warc path 并行解析数据
schema = StructType([
    StructField("value", StringType(), True),
])
page_content = sc.parallelize(warc_paths, len(warc_paths))
dump_html_df = page_content.mapPartitions(parse_path_to_html).toDF(schema)

# 写出s3

In [ ]:
config["skip_output_version"] = True
config["output_compression"] = "gz"
write_any_path(dump_html_df, output_path, config)